In [1]:
import ogb

/Users/javiprietod/Documents/1º Master/Geometria/Sheaf-NN/.venv/lib/python3.12/site-packages/outdated/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [2]:
from ogb.graphproppred import PygGraphPropPredDataset


In [3]:
dataset = PygGraphPropPredDataset(name='ogbg-molhiv')
print(dataset)

Downloaded 0.00 GB: 100%|██████████| 3/3 [00:04<00:00,  1.50s/it]
Processing...


Extracting dataset/hiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 41127/41127 [00:00<00:00, 225898.05it/s]


Converting graphs into PyG objects...


100%|██████████| 41127/41127 [00:00<00:00, 53982.77it/s]


Saving...
PygGraphPropPredDataset(41127)


Done!
/Users/javiprietod/Documents/1º Master/Geometria/Sheaf-NN/.venv/lib/python3.10/site-packages/ogb/graphproppred/dataset_pyg.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental fe